In [151]:
import math

In [34]:
# define a Vertex class
class Vertex:
    def __init__(self,node):
        self.id = node
        self.adjacent = {}
    def __str__(self):
        return str(self.id)+'adjacent:'+str([x.id for x in self.adjacent])
    def add_neighbor(self,neighbor,weight=0):
        self.adjacent[neighbor] = weight
    def get_connection(self):
        return self.adjacent.keys()
    def get_id(self):
        return self.id
    def get_weight(self,neighbor):
        return self.adjacent[neighbor]
# define a Graph class
class Graph:
    def __init__(self):
        self.vert_dict = {}
        self.number_vertices = 0
    def __iter__(self):
        # iterate through the Vertex Class object
        return iter(self.vert_dict.values())
    def add_vertex(self,node):
        self.number_vertices = self.number_vertices+1
        new_vertex = Vertex(node)
        self.vert_dict[node]  = new_vertex
        return new_vertex
    def add_edge(self,firstnode,secondnode,weight,directed = True):
        if firstnode not in self.vert_dict:
            self.add_vertex(firstnode)
        if secondnode not in self.vert_dict:
            self.add_vertex(secondnode)
        self.vert_dict[firstnode].add_neighbor(secondnode,weight)
        if not directed:
            self.vert_dict[secondnode].add_neighbor(firstnode,weight)
    def get_vertices(self):
        return self.vert_dict.keys()

In [299]:
def readgraph(filename):
    #function to read the graph from txt into list
    #store the raw information into raw
    with open(filename,'r') as file:
        contents = file.readlines()
    graph = Graph()
    for line in contents:
        content_list = line.split()
        graph.add_vertex(content_list[0])
        for connection in content_list[1:]:
            connection_list = connection.split(',')
            graph.add_edge(content_list[0],connection_list[0],int(connection_list[1]))
    return graph

def dijkstra(graph,source):
    # Dijkstra's shortest path algorithm in simple implementation
    X = [source]
    graph.vert_dict[source].shortest_path = 0
    while(len(X) < len(graph.get_vertices())):
        min_weight = float("infinity")
        for vertex_X in X:
            for vertex_notX in graph.vert_dict[vertex_X].get_connection():
                if(vertex_notX not in X):
                    weight = graph.vert_dict[vertex_X].shortest_path+graph.vert_dict[vertex_X].get_weight(vertex_notX)
                    if weight < min_weight:
                        min_weight = weight
                        min_target = vertex_notX
        X.append(min_target)
        graph.vert_dict[min_target].shortest_path = min_weight

def dijkstra_heap(graph,source):
    # Dijkstra's shortest path algorithm in heap implementation
    X = [source]
    graph.vert_dict[source].shortest_path = 0 
    heap = heap_init(graph,source)
    while(len(X)<len(graph.get_vertices())):
        (min_weight,min_target) = heap_getmin(heap,graph)
        heap_update(heap,min_weight,min_target,graph,X)
    return heap

def heap_init(graph,source):
    # Initialize heap
    heap = []
    for vertex in graph.vert_dict[source].get_connection():
        graph.vert_dict[vertex].shortest_path = graph.vert_dict[source].get_weight(vertex)
        heap_insert(heap,graph,vertex,len(heap))
    return heap
    
def heap_insert(heap,graph,vertex,i):
    # Insert vertex into heap based on the shortest_path
    heap.insert(i,vertex)
    # bubble up
    current_index = i
    graph.vert_dict[heap[current_index]].heap_location = current_index
    while(current_index>0 and graph.vert_dict[heap[current_index]].shortest_path<
          graph.vert_dict[heap[math.ceil(current_index/2)-1]].shortest_path):
        temp_vertex = heap[current_index]
        heap[current_index] = heap[math.ceil(current_index/2)-1]
        heap[math.ceil(current_index/2)-1] = temp_vertex
        # update the heap locaiton information in the graph
        graph.vert_dict[heap[math.ceil(current_index/2)-1]].heap_location = math.ceil(current_index/2)-1
        graph.vert_dict[heap[current_index]].heap_location = current_index
        current_index = math.ceil(current_index/2)-1
        
def heap_getmin(heap,graph):
    # Extract min from the heap
    min_target = heap[0]
    min_weight = graph.vert_dict[min_target].shortest_path
    final_elment = heap.pop()
    if(len(heap)>0):
        heap[0] = final_elment
    else:
        return min_weight,min_target
    # bubble down
    current_index = 0
    final_heap = len(heap)-1
    graph.vert_dict[heap[current_index]].heap_location = current_index
    temp_path = graph.vert_dict[heap[final_heap]].shortest_path
    while(current_index*2+1<=final_heap):
        if(current_index*2+2<=final_heap):
            #where the current node has two children
            if(graph.vert_dict[heap[current_index*2+1]].shortest_path<
              graph.vert_dict[heap[current_index*2+2]].shortest_path):
                min_child_path = graph.vert_dict[heap[current_index*2+1]].shortest_path
                min_child_index = current_index*2+1
            else:
                min_child_path = graph.vert_dict[heap[current_index*2+2]].shortest_path
                min_child_index = current_index*2+2
        else:
            #where the current node has one child
            min_child_path = graph.vert_dict[heap[current_index*2+1]].shortest_path
            min_child_index = current_index*2+1
        if(min_child_path<temp_path):
            temp_vertex = heap[current_index]
            heap[current_index] = heap[min_child_index]
            heap[min_child_index] = temp_vertex
            graph.vert_dict[heap[current_index]].heap_location = current_index
            graph.vert_dict[heap[min_child_index]].heap_location = min_child_index
            current_index = min_child_index
        else:
            break
    return min_weight,min_target
        
def heap_update(heap,minweight,changed_vertex,graph,X):
    # update the heap
    graph.vert_dict[changed_vertex].shortest_path = minweight
    X.append(changed_vertex)
    for influenced_vertex in graph.vert_dict[changed_vertex].get_connection():
        if(influenced_vertex not in X and influenced_vertex in heap):
            heap_location_old = graph.vert_dict[influenced_vertex].heap_location
            path_old = graph.vert_dict[heap[heap_location_old]].shortest_path
            path_new = minweight+graph.vert_dict[changed_vertex].get_weight(influenced_vertex)
            if (path_new<path_old):
                #if the new path is shorter, then we update it
                graph.vert_dict[influenced_vertex].shortest_path = path_new
                heap.pop(heap_location_old)
                heap_insert(heap,graph,influenced_vertex,heap_location_old)
        elif(influenced_vertex not in X and influenced_vertex not in heap):
            path_new = minweight+graph.vert_dict[changed_vertex].get_weight(influenced_vertex)
            graph.vert_dict[influenced_vertex].shortest_path = path_new
            heap_insert(heap,graph,influenced_vertex,len(heap))

In [310]:
#G = readgraph('dijkstra_data.txt')
G = readgraph('dijkstra_data.txt')
heap = dijkstra_heap(G,'1')
result = []
for vertex in range(1,11):
    result.append(G.vert_dict[str(vertex)].shortest_path)
result

[0, 2971, 2644, 3056, 2525, 2818, 2599, 1875, 745, 3205]

In [309]:
#G = readgraph('dijkstra_data.txt')
G = readgraph('dijkstra_data.txt')
heap = dijkstra(G,'1')
result = []
for vertex in range(1,11):
    result.append(G.vert_dict[str(vertex)].shortest_path)
result

[0, 2971, 2644, 3056, 2525, 2818, 2599, 1875, 745, 3205]

In [311]:
target = [7,37,59,82,99,115,133,165,188,197]
result = []
for vertex in target:
    result.append(G.vert_dict[str(vertex)].shortest_path)
result

[2599, 2610, 2947, 2052, 2367, 2399, 2029, 2442, 2505, 3068]